In [1]:
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import os
import glob

In [2]:
os.chdir("export")
images = glob.glob("*.jpeg")
xml_mask = glob.glob("*.xml")

In [3]:
classes = {
  "tree":(1,1,1),
  "landscaping":(2,2,2),
  "road":(3,3,3),
  "sidewalk":(4,4,4),
  "building":(5,5,5),
  "sky":(6,6,6),
  "concrete":(7,7,7)
}


In [4]:
for mask in xml_mask:
    
    root = ET.parse(mask).getroot()

    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    img = Image.new("RGB", (width*2, height*2), 0)

    for objects in root.findall("object"):

      object_class = classes[objects.find("name").text]

      for object in objects.findall('polygon'):
        points=[]
        for element in object.findall('*'):
            points.append(element.tag)
        
        polygon = []        
        for point in range(0, len(points) - 1, 2):
            x = int(object.find(points[point]).text)*2
            y = int(object.find(points[point + 1]).text)*2
            point = (x,y)
            polygon.append(point)
        ImageDraw.Draw(img).polygon(polygon, fill=object_class,outline=object_class)

        
    imagepath = "masks"
    
    try: 
        os.makedirs(imagepath)
    except OSError:
        if not os.path.isdir(imagepath):
            raise
    img = img.resize((width, height), Image.ANTIALIAS)
    img.save(imagepath+"/"+mask.split(".")[0]+".png","PNG")